In [102]:
import os
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection  import train_test_split

## Input from XgB, RF and GBM

In [78]:
XgB_Train_Pred = pd.read_csv("C:/Users/KevinZhang/Documents/GitHub/Group20-Project/Output/XgB_output/XgB_Train_Data_Pred.csv",index_col=0)
RF_Train_Pred = pd.read_csv("C:/Users/KevinZhang/Documents/GitHub/Group20-Project/Output/RF_output/RF_Train_Data_Pred.csv",index_col=0)
GBM_Train_Pred = pd.read_csv("C:/Users/KevinZhang/Documents/GitHub/Group20-Project/Output/GBM_output/GBM_Train_Data_Pred.csv",sep='\t',index_col=0)

In [79]:
# Checking the id
print(sum(XgB_Train_Pred['id'] == RF_Train_Pred['ID']) == XgB_Train_Pred.shape[0])
print(sum(RF_Train_Pred['ID'] == GBM_Train_Pred['id']) == XgB_Train_Pred.shape[0])

True
True


In [84]:
Train_Pred = pd.concat([XgB_Train_Pred['Prediction'], RF_Train_Pred['Prediction'], GBM_Train_Pred['predictions']], axis=1)
Train_Pred.columns = ['XgB_Train_Pred', 'RF_Train_Pred', 'GBM_Train_Pred']
Train_Pred.shape

(28734, 3)

In [81]:
XgB_Test_Pred = pd.read_csv("C:/Users/KevinZhang/Documents/GitHub/Group20-Project/Output/XgB_output/XgB_Test_Data_Pred.csv",index_col=0)
RF_Test_Pred = pd.read_csv("C:/Users/KevinZhang/Documents/GitHub/Group20-Project/Output/RF_output/RF_Test_Data_Pred.csv",index_col=0)
GBM_Test_Pred = pd.read_csv("C:/Users/KevinZhang/Documents/GitHub/Group20-Project/Output/GBM_output/GBM_Test_Data_Pred.csv",sep='\t',index_col=0)

In [82]:
# Checking the id
print(sum(XgB_Test_Pred['id'] == RF_Test_Pred['ID']) == XgB_Test_Pred.shape[0])
print(sum(RF_Test_Pred['ID'] == GBM_Test_Pred['id']) == XgB_Test_Pred.shape[0])

True
True


In [185]:
Test_Pred = pd.concat([XgB_Test_Pred['Prediction'], RF_Test_Pred['Prediction'], GBM_Test_Pred['predictions']], axis=1)
Test_Pred.columns = ['XgB_Test_Pred', 'RF_Test_Pred', 'GBM_Test_Pred']
Test_Pred.shape

(9581, 3)

In [193]:
All_Data_ID = pd.DataFrame(data =XgB_Train_Pred['id'])
All_Data_ID = All_Data_ID.append(pd.DataFrame(data = XgB_Test_Pred['id']), ignore_index=True)
All_Data_ID

,id
0,6374694
1,7218561
2,20553159
3,17415890
4,16825557
5,22689704
6,22807347
7,10268868
8,17972013
9,26110466


## Processing Train and Test Data

In [88]:
Train_Data = pd.read_csv("C:/Users/KevinZhang/Documents/GitHub/Group20-Project/Output/train.csv")
Test_Data = pd.read_csv("C:/Users/KevinZhang/Documents/GitHub/Group20-Project/Output/test.csv")
Train_Y = Train_Data.pop('review_scores_rating')
Test_Y = Test_Data.pop('review_scores_rating')
print(Train_Y.shape)
print(Test_Y.shape)

(28734,)
(9581,)


## Linear Regression Ensemble

In [100]:
# Create linear regression object
Linear_Reg_Ensemble = linear_model.LinearRegression()

# Train the model using the training sets
Linear_Reg_Ensemble.fit(Train_Pred, Train_Y)

# Make predictions using the testing set
Linear_Ensemble_Pred = Linear_Reg_Ensemble.predict(Test_Pred)

# The coefficients
print('Coefficients: \n', Linear_Reg_Ensemble.coef_)
# The RMSE
print("RMSE: %f" % (np.sqrt(mean_squared_error(Linear_Ensemble_Pred, Test_Y))))
# Explained variance score: 1 is perfect prediction
print('R_Squared: %.2f' % r2_score(Test_Y, Linear_Ensemble_Pred))

Coefficients: 
 [0.26600236 1.13961649 0.24619432]
RMSE: 9.628552
R_Squared: 0.06


## Lasso Regression Ensemble

In [169]:
alpha_list = [0.01, 0.1, 0.3, 0.5, 0.7, 1, 3, 5, 10]
RMAE_LASSO = []
for i in range(9):
    X_train,X_test,y_train,y_test = train_test_split(Train_Pred, Train_Y, test_size=0.2)
    alpha = alpha_list[i]
    LASSO_Ensemble = linear_model.Lasso(alpha=alpha, max_iter=10e5)
    LASSO_Ensemble.fit(X_train,y_train)
    LASSO_Ensemble_Pred = LASSO_Ensemble.predict(X_test)
    RMAE_LASSO.append(np.sqrt(mean_squared_error(y_test, LASSO_Ensemble_Pred)))
Optimal_alpha = alpha_list[RMAE_LASSO.index(min(RMAE_LASSO))]

In [170]:
Optimal_alpha

3

In [171]:
LASSO_Ensemble = linear_model.Lasso(alpha=Optimal_alpha, max_iter=10e5)
LASSO_Ensemble.fit(Train_Pred,Train_Y)
LASSO_Ensemble_Pred = LASSO_Ensemble.predict(Test_Pred)
print("RMSE: %f" % (np.sqrt(mean_squared_error(LASSO_Ensemble_Pred, Test_Y))))

RMSE: 9.526082


In [174]:
LASSO_Ensemble.coef_

array([0.44776132, 0.76354252, 0.05537988])

In [181]:
Train_Pred.columns = ['XgB_Pred', 'RF_Pred', 'GBM_Pred']
Test_Pred.columns = ['XgB_Pred', 'RF_Pred', 'GBM_Pred']
Train_Test_Pred = Train_Pred.append(Test_Pred)
Train_Test_Pred

,XgB_Pred,RF_Pred,GBM_Pred
0,91.694760,92.168051,92.773796
1,96.525190,96.851028,95.967843
2,94.039620,93.198758,94.704260
3,96.007385,94.103380,95.201510
4,93.279366,92.411390,94.491005
5,92.358800,92.760375,92.560979
6,93.784670,93.460515,93.907224
7,96.448870,95.717467,95.125854
8,91.844140,93.021977,92.298198
9,96.495930,94.551132,97.894611


In [220]:
LASSO_Ensemble_All_Data_Pred = LASSO_Ensemble.predict(Train_Test_Pred)
Actual_Rating = Train_Y.append(Test_Y)
Actual_Rating = pd.DataFrame(Actual_Rating)

In [225]:
All_Data_Pred = pd.concat([pd.DataFrame(All_Data_ID['id']), pd.DataFrame(LASSO_Ensemble_All_Data_Pred,columns=['Ensemble_Prediction'])], axis=1)

In [231]:
All_Data_Pred.to_csv('All_Data_Pred.csv')